<a href="https://colab.research.google.com/github/lin3372/400_gan/blob/main/books/generative_deep_learning/0301_gan_autoencoder_train_(iL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup 

In [41]:
GOOGLE_COLAB=0
USE_GPU=0

## gpu

In [42]:
try:
  !nvidia-smi
except:
  print("couldn not communicate with the NVIDIA driver")

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  #raise SystemError('GPU device not found')
  print('GPU device not found, using CPU')
else:
  USE_GPU=1
  print('Found GPU at: {}'.format(device_name))


Fri Jan 14 07:52:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    31W /  70W |   1172MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## mount google drive



In [43]:
import os, sys
if 'google.colab' in sys.modules:
  print("Running in Google Colab, mounting GDrive")
  from google.colab import drive
  drive.mount('/gdrive')
  GOOGLE_COLAB=1

  PROJ_DATA_DIR = "/gdrive/MyDrive/github/400_gan/books/generative_deep_learning/"
  os.makedirs(PROJ_DATA_DIR, exist_ok=True)


Running in Google Colab, mounting GDrive
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## working directory setting

1. With Colab this prog will clone from davidADSP/GDL_code.git
2. With local machine this prog will be running in the directory and the cloned repo will be in "./src/"

 directory
* The **data** folder is where to download relevant data sources (chapter 3 onwards)
* The **run** folder stores output from the generative models (chapter 3 onwards) 
* The **utils** folder stores useful functions that are sourced by the main notebooks

In [44]:
if (GOOGLE_COLAB==1): ## when executing from PC
  PROJ_WORKING_DIR = "/gdrive/MyDrive/github/400_gan/books/generative_deep_learning/src/"
  PROJ_LOG_PATH=PROJ_WORKING_DIR + "/log/data_0301/"
else:  ## when executing from PC, assuming running from where the inpynb is located
  PROJ_WORKING_DIR = "./src/"
  PROJ_LOG_PATH=PROJ_DAT_PATH + "/log/data_0301/"

os.makedirs(PROJ_WORKING_DIR, exist_ok=True)  
os.makedirs(PROJ_LOG_PATH, exist_ok=True) 

## Clone/copy from Github/GDrive

In [45]:
import os
import shutil 
import sys 

!git config --global user.email "lin3372@gmail.com"
!git config --global user.name "lin3372"

if os.path.exists("./400_gan/") and os.path.isdir("./400_gan/"):
  shutil.rmtree("./400_gan/")
if os.path.exists("./src/") and os.path.isdir("./src/"):
  shutil.rmtree("./src/")

if (GOOGLE_COLAB==1):
  try:
    !git clone https://github.com/lin3372/400_gan 
    #!git clone https://github.com/davidADSP/GDL_code.git
  except:
    print("davidADSP/GDL_code clone failed!")      
  else:
    print("davidADSP/GDL_code cloned!")

  PROJ_GIT_SRC_PATH="./400_gan/books/generative_deep_learning/src/"
else: # running on PC
  PROJ_DAT_PATH="./src/"

shutil.copytree(PROJ_GIT_SRC_PATH, "./src")

os.chdir("./src/")

Cloning into '400_gan'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 189 (delta 86), reused 106 (delta 37), pack-reused 0
Receiving objects: 100% (189/189), 15.23 MiB | 39.59 MiB/s, done.
Resolving deltas: 100% (86/86), done.
davidADSP/GDL_code cloned!


# Autoencoder

In [46]:
import os
import tensorflow as tf
import keras

from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras.utils import to_categorical

from utils.loaders import load_mnist
from models.AE import Autoencoder

## Set parameters

In [47]:
# run params
SECTION = 'vae'
RUN_ID = '0001'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

MODE =  'build' #'load' #

## Load the data

In [48]:
(x_train, y_train), (x_test, y_test) = load_mnist()
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

## Define the structure of the neural network

In [49]:
AE = Autoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if MODE == 'build':
    AE.save(RUN_FOLDER)
else:
    AE.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [50]:
AE.encoder.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder_conv_0 (Conv2D)     (None, 28, 28, 32)        320       
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 28, 28, 32)        0         
                                                                 
 encoder_conv_1 (Conv2D)     (None, 14, 14, 64)        18496     
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 encoder_conv_2 (Conv2D)     (None, 7, 7, 64)          36928     
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 7, 7, 64)          0   

In [51]:
AE.decoder.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 3136)              9408      
                                                                 
 reshape_1 (Reshape)         (None, 7, 7, 64)          0         
                                                                 
 decoder_conv_t_0 (Conv2DTra  (None, 7, 7, 64)         36928     
 nspose)                                                         
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 7, 7, 64)          0         
                                                                 
 decoder_conv_t_1 (Conv2DTra  (None, 14, 14, 64)       36928     
 nspose)                                                   

## Train the autoencoder

In [52]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0

In [53]:
AE.compile(LEARNING_RATE)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [54]:
AE.train(     
    x_train[:1000]
    , batch_size = BATCH_SIZE
    , epochs = 200
    , run_folder = RUN_FOLDER
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
30/32 [===========================>..] - ETA: 0s - loss: 0.1817
Epoch 00001: saving model to run/vae/0001_digits/weights/weights.h5
32/32 [==============================] - 3s 76ms/step - loss: 0.1778 - lr: 5.0000e-04
Epoch 2/200
24/32 [=====================>........] - ETA: 0s - loss: 0.0878
Epoch 00002: saving model to run/vae/0001_digits/weights/weights.h5
32/32 [==============================] - 0s 8ms/step - loss: 0.0837 - lr: 5.0000e-04
Epoch 3/200
28/32 [=========================>....] - ETA: 0s - loss: 0.0679
Epoch 00003: saving model to run/vae/0001_digits/weights/weights.h5
32/32 [==============================] - 0s 6ms/step - loss: 0.0679 - lr: 5.0000e-04
Epoch 4/200
28/32 [=========================>....] - ETA: 0s - loss: 0.0639
Epoch 00004: saving model to run/vae/0001_digits/weights/weights.h5
32/32 [==============================] - 0s 6ms/step - loss: 0.0637 - lr: 5.0000e-04
Epoch 5/200
27/32 [========================>.....] - ETA: 0s - loss: 0.0592
Epoch 0

In [71]:
GDRIVE_LOG_FOLDER = PROJ_WORKING_DIR + RUN_FOLDER
os.makedirs(GDRIVE_LOG_FOLDER, exist_ok=True)  
os.system(f"cp -R {RUN_FOLDER}/* {GDRIVE_LOG_FOLDER}")

0